In [1]:
import os
import numpy as np
import cv2 as cv
import dlib
%matplotlib inline
import matplotlib.pyplot as plt
face_detector = dlib.get_frontal_face_detector()
main_path='dataset/gender/'
predictor_model="shape_predictor_68_face_landmarks copy.dat"
predictor = dlib.shape_predictor(predictor_model)
facerec=dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
d=os.listdir(main_path)
img_array=[]
img_labels=[]
i=0
for i,dirs in enumerate(sorted(d)):
    if(dirs.startswith('ksp')):
        print(dirs)
        for item in (os.listdir(main_path+dirs)):
            label=[0,0]
            if(item.endswith('.jpg')):
                img=cv.imread(main_path+dirs+'/'+item)
                gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                
                face_rect= face_detector(gray, 1)
                
                
                for k,d in enumerate(face_rect):
                    
                    #x = d.left()
                    #y = d.top()
                    #w = d.right()-x
                    #h = d.bottom()-y
                    shape=predictor(gray,d)
                    face_descriptor = facerec.compute_face_descriptor(img,shape )
                #rect=cv.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                #rect = img[y:y+h,x:x+w]
                label[i-1]=1
                img_array.append(face_descriptor)
                img_labels.append(label)
                    
                    
                #face_rect = face_detector(img, 1)

img_dataset=np.array(img_array)
img_labell=np.array(img_labels)
print(img_dataset.shape)
print(img_labell.shape)

ksp_adult_male
ksp_adult_woman
(78, 128)
(78, 2)


In [2]:
img_labell[24]

array([1, 0])

In [3]:
#plt.imshow(img_dataset[24])
print(img_dataset[25].shape)

(128,)


In [4]:
image_size=128
def reformat(dataset):
    dataset=dataset.reshape((-1,1*image_size)).astype(np.float32)
    return dataset
train_dataset=reformat(img_dataset)
train_labels=img_labell
print(train_dataset.shape)
print(train_labels.shape)

(78, 128)
(78, 2)


In [5]:
import tensorflow as tf
#train_subset=105
graph=tf.Graph()
hidden_nodes1=80
hidden_nodes2=50


batch_size=30
image_size=128
num_classes=train_labels.shape[1]
depth=10
with graph.as_default():
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,1*image_size))
    tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,num_classes))
    
    
    #tf_valid_dataset=tf.constant(valid_dataset)
    #tf_test_dataset=tf.constant(test_dataset)
    
    weights_l1=tf.Variable(tf.truncated_normal([1*image_size,hidden_nodes1]))
    biases_l1=tf.Variable(tf.zeros([hidden_nodes1]))
    weights_l2=tf.Variable(tf.truncated_normal([hidden_nodes1,hidden_nodes2]))
    biases_l2=tf.Variable(tf.zeros([hidden_nodes2]))
    weights_l3=tf.Variable(tf.truncated_normal([hidden_nodes2,num_classes]))
    biases_l3=tf.Variable(tf.zeros([num_classes]))
    global_step=tf.Variable(0)
    learning_rate =tf.train.exponential_decay(0.05, global_step,100, 0.9, staircase=True)
    saver = tf.train.Saver(write_version=tf.train.SaverDef.V1)#
    
 #graph structure   
    def computation(dataset):
        logits_1=tf.matmul(dataset,weights_l1)+biases_l1
        logits_1=tf.nn.relu(logits_1)
        keep_prob1=tf.placeholder(tf.float32)
        drop_out=tf.nn.dropout(logits_1,keep_prob1)
        logits_2=tf.matmul(logits_1,weights_l2)+biases_l2
       # keep_prob=tf.placeholder(tf.float32)
        #drop_out=tf.nn.dropout(logits_2,keep_prob)
        logits_2=tf.nn.relu(logits_2)
        logits_3=tf.matmul(logits_2,weights_l3)+biases_l3
        
        return logits_3
        
        
        
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=computation(tf_train_dataset),labels=tf_train_labels)+0.01*(tf.nn.l2_loss(weights_l1)+tf.nn.l2_loss(biases_l1)+tf.nn.l2_loss(weights_l2)
                                                                                                                              +tf.nn.l2_loss(biases_l2)+tf.nn.l2_loss(weights_l3)+tf.nn.l2_loss(biases_l3)))          
    
    optimizer=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)#,global_step=global_step)
    #optimizer=tf.train.AdamOptimizer(learning_rate).minimize(loss)#,global_step=global_step)
    train_prediction=tf.nn.softmax(computation(tf_train_dataset))
    



/Users/avisheksarkar/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [6]:
num_steps=500
alpha=['male','woman']
def accuracy(prediction,labels):
    return (100.0*np.sum(np.argmax(prediction,1)==np.argmax(labels,1)))/prediction.shape[0]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('initialized')
    
    for step in range(num_steps):
        offset=(step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_dataset=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        feed_dict={tf_train_dataset:batch_dataset, tf_train_labels:batch_labels}
       
    
        _,l,p=session.run([optimizer,loss,train_prediction,],feed_dict=feed_dict)
        if(step%100==0):
            print("loss at ",step,':',l)
            print('training accuracy :',accuracy(p,batch_labels))
    saver.save(session,'models/test_model2_gender.ckpt')        
    cap = cv.VideoCapture(0)
    ret,frame=cap.read()
    while ret:
        ret, frame = cap.read()
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        #gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        detected_faces = face_detector(gray, 1)
        for k,d in enumerate(detected_faces):
            shape=predictor(gray,d)
            face_descriptor = facerec.compute_face_descriptor(frame,shape )
            image_data1=np.array(face_descriptor,dtype=np.float32)
            img=image_data1.reshape(1,1*128)
            print(tf.nn.softmax(computation(img)).eval())
            test=tf.nn.softmax(computation(img)).eval()
            index=np.argmax(test)
            print("predicted class is: ",alpha[index])

initialized
loss at  0 : 61.270653
training accuracy : 40.0
loss at  100 : 49.266445
training accuracy : 100.0
loss at  200 : 44.58272
training accuracy : 100.0
loss at  300 : 40.33759
training accuracy : 100.0
loss at  400 : 36.506573
training accuracy : 100.0
[[0.8301742  0.16982576]]
predicted class is:  male
[[0.56535906 0.43464088]]
predicted class is:  male
[[0.4728947  0.52710533]]
predicted class is:  woman
[[0.9784073  0.02159269]]
predicted class is:  male
[[0.95153093 0.04846912]]
predicted class is:  male
[[0.8080397 0.1919603]]
predicted class is:  male
[[0.9726954  0.02730452]]
predicted class is:  male


KeyboardInterrupt: 